In [ ]:
import yt_dlp

In [14]:
import yt_dlp
import os

# FALLBACK HIGH-QUALITY DOWNLOADS (No FFmpeg required)

def download_best_single_format(url, output_path='downloads'):
    """
    Download best quality without needing FFmpeg (single format file)
    """
    # Create downloads directory
    os.makedirs(output_path, exist_ok=True)
    
    ydl_opts = {
        # This selects the best single file that contains both video and audio
        'format': 'best[ext=mp4]/best[ext=webm]/best',
        'outtmpl': os.path.join(output_path, '%(title)s.%(ext)s'),
        'writeinfojson': True,
        # Don't write subtitle files directly - we'll process them programmatically
        'writesubtitles': False,
        'writeautomaticsub': False,
    }
    
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=False)
            print(f"Title: {info.get('title', 'Unknown')}")
            
            # Find the best single format
            formats = info.get('formats', [])
            single_formats = [f for f in formats if f.get('vcodec', 'none') != 'none' and f.get('acodec', 'none') != 'none']
            
            if single_formats:
                best_single = max(single_formats, key=lambda x: (x.get('height', 0), x.get('width', 0)))
                print(f"Selected format: {best_single.get('height', 'Unknown')}p ({best_single.get('ext', 'Unknown')})")
            
            print("Downloading (no FFmpeg needed)...")
            ydl.download([url])
            
            # Extract and save transcript with timestamps
            transcript_path = extract_transcript_with_timestamps(info, output_path)
            if transcript_path:
                print(f"Transcript saved to: {transcript_path}")
            
            print("Download completed!")
            
            return info
            
    except Exception as e:
        print(f"Error downloading: {str(e)}")
        return None

def download_quality_no_ffmpeg(url, max_height=1080, output_path='downloads'):
    """
    Download specific quality without FFmpeg requirement
    """
    # Create downloads directory
    os.makedirs(output_path, exist_ok=True)
    
    ydl_opts = {
        # Select best single format up to specified height
        'format': f'best[height<={max_height}][ext=mp4]/best[height<={max_height}][ext=webm]/best[height<={max_height}]',
        'outtmpl': os.path.join(output_path, '%(title)s_%(height)sp.%(ext)s'),
        'writeinfojson': True,
        # Don't write subtitle files directly - we'll process them programmatically
        'writesubtitles': False,
        'writeautomaticsub': False,
    }
    
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=False)
            print(f"Title: {info.get('title', 'Unknown')}")
            print(f"Requesting: up to {max_height}p (single format)")
            
            ydl.download([url])
            
            # Extract and save transcript with timestamps
            transcript_path = extract_transcript_with_timestamps(info, output_path)
            if transcript_path:
                print(f"Transcript saved to: {transcript_path}")
            
            print(f"Download completed!")
            
            return info
            
    except Exception as e:
        print(f"Error downloading: {str(e)}")
        return None

def extract_transcript_with_timestamps(info, output_path):
    """
    Extract transcript with timestamps from video info and save as .txt
    """
    try:
        title = info.get('title', 'Unknown_Video')
        # Clean title for filename
        clean_title = "".join(c for c in title if c.isalnum() or c in (' ', '-', '_')).rstrip()
        transcript_filename = f"{clean_title}_transcript.txt"
        transcript_path = os.path.join(output_path, transcript_filename)
        
        # Try to get subtitles from the info
        subtitles = info.get('subtitles', {})
        automatic_captions = info.get('automatic_captions', {})
        
        # Look for English subtitles first, then auto-generated
        subtitle_data = None
        if 'en' in subtitles:
            subtitle_data = subtitles['en']
        elif 'en' in automatic_captions:
            subtitle_data = automatic_captions['en']
        
        if subtitle_data:
            # Find VTT format subtitles
            vtt_subtitle = None
            for sub in subtitle_data:
                if sub.get('ext') == 'vtt':
                    vtt_subtitle = sub
                    break
            
            if vtt_subtitle and 'url' in vtt_subtitle:
                # Download and parse VTT file
                import urllib.request
                
                try:
                    with urllib.request.urlopen(vtt_subtitle['url']) as response:
                        vtt_content = response.read().decode('utf-8')
                    
                    # Parse VTT and convert to readable transcript
                    transcript_text = parse_vtt_to_transcript(vtt_content)
                    
                    # Save to file
                    with open(transcript_path, 'w', encoding='utf-8') as f:
                        f.write(f"Video Title: {title}\n")
                        f.write(f"URL: {info.get('webpage_url', 'Unknown')}\n")
                        f.write(f"Duration: {info.get('duration', 'Unknown')} seconds\n")
                        f.write("=" * 50 + "\n\n")
                        f.write(transcript_text)
                    
                    return transcript_path
                    
                except Exception as e:
                    print(f"Error downloading transcript: {str(e)}")
        
        # If no subtitles available, create a placeholder
        with open(transcript_path, 'w', encoding='utf-8') as f:
            f.write(f"Video Title: {title}\n")
            f.write(f"URL: {info.get('webpage_url', 'Unknown')}\n")
            f.write(f"Duration: {info.get('duration', 'Unknown')} seconds\n")
            f.write("=" * 50 + "\n\n")
            f.write("No transcript available for this video.\n")
            f.write("Either subtitles are disabled or not generated for this content.")
        
        return transcript_path
        
    except Exception as e:
        print(f"Error creating transcript file: {str(e)}")
        return None

def parse_vtt_to_transcript(vtt_content):
    """
    Parse VTT subtitle content and convert to readable transcript with timestamps
    """
    lines = vtt_content.split('\n')
    transcript = []
    current_time = ""
    current_text = ""
    
    for line in lines:
        line = line.strip()
        
        # Skip VTT headers and empty lines
        if line.startswith('WEBVTT') or line.startswith('NOTE') or not line:
            continue
        
        # Check if line contains timestamp (format: 00:00:00.000 --> 00:00:00.000)
        if '-->' in line:
            # Save previous entry if exists
            if current_time and current_text:
                transcript.append(f"[{current_time}] {current_text}")
            
            # Extract start time
            current_time = line.split(' --> ')[0]
            current_text = ""
        else:
            # This is subtitle text
            if current_text:
                current_text += " " + line
            else:
                current_text = line
    
    # Add the last entry
    if current_time and current_text:
        transcript.append(f"[{current_time}] {current_text}")
    
    return '\n'.join(transcript)

# Check if FFmpeg is available
def check_ffmpeg():
    """Check if FFmpeg is installed"""
    import subprocess
    try:
        result = subprocess.run(['ffmpeg', '-version'], capture_output=True, text=True)
        if result.returncode == 0:
            print("✅ FFmpeg is installed and working!")
            return True
        else:
            print("❌ FFmpeg not found in PATH")
            return False
    except FileNotFoundError:
        print("❌ FFmpeg is not installed")
        return False

# Test FFmpeg availability
print("=== Checking FFmpeg availability ===")
ffmpeg_available = check_ffmpeg()

if ffmpeg_available:
    print("\n🎉 You can use the high-quality download functions!")
else:
    print("\n⚠️  Use the fallback functions below instead:")
    print("   - download_best_single_format(url)")
    print("   - download_quality_no_ffmpeg(url, 1080)")

# Example usage with fallback (works without FFmpeg)
test_url = "https://youtu.be/kopoLzvh5jY?si=CwgrjehubePXI_Uw"

print(f"\n=== Testing fallback download (no FFmpeg needed) ===")
# Uncomment to download:
# download_best_single_format(test_url)

print(f"\n=== Alternative: Specific quality without FFmpeg ===") 
# Uncomment to download up to 1080p:
# download_quality_no_ffmpeg(test_url, 1080)

=== Checking FFmpeg availability ===
✅ FFmpeg is installed and working!

🎉 You can use the high-quality download functions!

=== Testing fallback download (no FFmpeg needed) ===

=== Alternative: Specific quality without FFmpeg ===


In [ ]:
import yt_dlp
import os

# Advanced download options
def download_video_advanced(url, output_path='downloads', quality='best', audio_only=False):
    """
    Advanced video download with more options
    
    Args:
        url (str): YouTube video URL
        output_path (str): Directory to save the video
        quality (str): Video quality ('best', 'worst', '720p', etc.)
        audio_only (bool): Download only audio
    """
    # Create downloads directory
    os.makedirs(output_path, exist_ok=True)
    
    if audio_only:
        # Audio-only download options
        ydl_opts = {
            'format': 'bestaudio/best',
            'outtmpl': os.path.join(output_path, '%(title)s.%(ext)s'),
            'writeinfojson': True,
            # Don't write subtitle files directly - we'll process them programmatically
            'writesubtitles': False,
            'writeautomaticsub': False,
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192',
            }],
        }
    else:
        # Video download options
        format_selector = {
            'best': 'best[height<=1080]',
            'worst': 'worst',
            '720p': 'best[height<=720]',
            '480p': 'best[height<=480]',
            '360p': 'best[height<=360]',
        }
        
        ydl_opts = {
            'format': format_selector.get(quality, 'best[height<=1080]'),
            'outtmpl': os.path.join(output_path, '%(title)s.%(ext)s'),
            'writeinfojson': True,
            # Don't write subtitle files directly - we'll process them programmatically
            'writesubtitles': False,
            'writeautomaticsub': False,
        }
    
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=False)
            print(f"Title: {info.get('title', 'Unknown')}")
            print(f"Duration: {info.get('duration', 'Unknown')} seconds")
            
            if audio_only:
                print("Downloading audio only...")
            else:
                print(f"Downloading video in {quality} quality...")
            
            ydl.download([url])
            
            # Extract and save transcript with timestamps
            transcript_path = extract_transcript_with_timestamps(info, output_path)
            if transcript_path:
                print(f"Transcript saved to: {transcript_path}")
            
            print("Download completed!")
            
            return info
            
    except Exception as e:
        print(f"Error downloading: {str(e)}")
        return None

# Example usage:
# download_video_advanced(youtube_url, quality='720p')  # Download in 720p
# download_video_advanced(youtube_url, audio_only=True)  # Download audio only

In [4]:
# Utility functions for video information
def get_video_info(url):
    """
    Get detailed information about a video without downloading
    """
    ydl_opts = {
        'quiet': True,  # Suppress output
    }
    
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=False)
            
            print("=== Video Information ===")
            print(f"Title: {info.get('title', 'N/A')}")
            print(f"Uploader: {info.get('uploader', 'N/A')}")
            print(f"Duration: {info.get('duration', 0)} seconds ({info.get('duration', 0) // 60} minutes)")
            print(f"View count: {info.get('view_count', 'N/A'):,}")
            print(f"Upload date: {info.get('upload_date', 'N/A')}")
            print(f"Description: {info.get('description', 'N/A')[:100]}...")
            
            return info
            
    except Exception as e:
        print(f"Error getting video info: {str(e)}")
        return None

def list_available_formats(url):
    """
    List all available download formats for a video
    """
    ydl_opts = {
        'listformats': True,  # List formats only
        'quiet': True,
    }
    
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=False)
            
            print("=== Available Formats ===")
            formats = info.get('formats', [])
            
            for fmt in formats[-10:]:  # Show last 10 formats (usually best quality)
                format_id = fmt.get('format_id', 'N/A')
                ext = fmt.get('ext', 'N/A')
                resolution = fmt.get('resolution', 'N/A')
                filesize = fmt.get('filesize')
                filesize_str = f"{filesize // (1024*1024)} MB" if filesize else "Unknown size"
                
                print(f"ID: {format_id} | Extension: {ext} | Resolution: {resolution} | Size: {filesize_str}")
            
            return formats
            
    except Exception as e:
        print(f"Error listing formats: {str(e)}")
        return None

# Example usage:
# get_video_info(youtube_url)
# list_available_formats(youtube_url)

## Quick Reference Guide

### Basic Usage
1. **Install**: Run the first cell to install `yt-dlp`
2. **Simple Download**: Use `download_video(url)` for basic video download
3. **Get Info**: Use `get_video_info(url)` to see video details without downloading

### Common Format Options
- `'best'` - Highest quality available
- `'worst'` - Lowest quality available  
- `'best[height<=720]'` - Best quality up to 720p
- `'bestaudio'` - Audio only, best quality
- `'best[ext=mp4]'` - Best quality in MP4 format

### Tips
- Always create a downloads directory first
- Check video info before downloading large files
- Use lower quality settings for playlists to save space
- Audio-only downloads are much smaller and faster

### Troubleshooting
- If download fails, try updating yt-dlp: `!pip install --upgrade yt-dlp`
- Some videos may be region-locked or age-restricted
- Private or deleted videos cannot be downloaded

In [15]:
# HIGH QUALITY DOWNLOAD - Best possible quality
def download_highest_quality(url, output_path='downloads'):
    """
    Download video in the absolute highest quality available
    """
    
    ydl_opts = {
        # This format string prioritizes:
        # 1. Best video quality available
        # 2. Best audio quality available  
        # 3. Combines them if they're separate streams
        'format': 'bestvideo+bestaudio/best',
        
        # Alternative high-quality formats to try
        'format_selector': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/bestvideo+bestaudio/best[ext=mp4]/best',
        
        'outtmpl': os.path.join(output_path, '%(title)s.%(ext)s'),
        'writeinfojson': True,
        # Don't write subtitle files directly - we'll process them programmatically
        'writesubtitles': False,
        'writeautomaticsub': False,
        
        # Merge video and audio into single file if needed
        'merge_output_format': 'mp4',
        
        # Post-processing options
        'postprocessors': [{
            'key': 'FFmpegVideoConvertor',
            'preferedformat': 'mp4',
        }],
    }
    
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=False)
            print(f"Title: {info.get('title', 'Unknown')}")
            print(f"Duration: {info.get('duration', 'Unknown')} seconds")
            
            # Show available formats
            formats = info.get('formats', [])
            if formats:
                best_video = max((f for f in formats if f.get('vcodec') != 'none'), 
                               key=lambda x: (x.get('height', 0), x.get('width', 0)), default=None)
                best_audio = max((f for f in formats if f.get('acodec') != 'none'), 
                               key=lambda x: x.get('abr', 0), default=None)
                
                if best_video:
                    print(f"Best video: {best_video.get('height', 'Unknown')}p @ {best_video.get('fps', 'Unknown')}fps")
                if best_audio:
                    print(f"Best audio: {best_audio.get('abr', 'Unknown')} kbps")
            
            print("\nDownloading in highest quality...")
            ydl.download([url])
            
            # Extract and save transcript with timestamps
            transcript_path = extract_transcript_with_timestamps(info, output_path)
            if transcript_path:
                print(f"Transcript saved to: {transcript_path}")
                
            print("High-quality download completed!")
            
            return info
            
    except Exception as e:
        print(f"Error downloading: {str(e)}")
        return None

# SPECIFIC QUALITY DOWNLOAD - Choose exact resolution
def download_specific_quality(url, height=1080, output_path='downloads'):
    """
    Download video in specific quality (height in pixels) with transcript
    
    Args:
        url: YouTube URL
        height: Video height (e.g., 720, 1080, 1440, 2160 for 4K)
        output_path: Download directory
    """
    
    # Create downloads directory
    os.makedirs(output_path, exist_ok=True)
    
    # Format selection prioritizes the requested height
    format_string = f'bestvideo[height<={height}]+bestaudio/best[height<={height}]'
    
    ydl_opts = {
        'format': format_string,
        'outtmpl': os.path.join(output_path, '%(title)s_%(height)sp.%(ext)s'),
        'writeinfojson': True,
        # Don't write subtitle files directly - we'll process them programmatically
        'writesubtitles': False,
        'writeautomaticsub': False,
        'merge_output_format': 'mp4',
    }
    
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=False)
            print(f"Title: {info.get('title', 'Unknown')}")
            print(f"Requesting: {height}p quality")
            
            ydl.download([url])
            
            # Extract and save transcript with timestamps
            transcript_path = extract_transcript_with_timestamps(info, output_path)
            if transcript_path:
                print(f"Transcript saved to: {transcript_path}")
            
            print(f"Download completed in {height}p!")
            
            return info
            
    except Exception as e:
        print(f"Error downloading: {str(e)}")
        return None

# Test with your URL - try highest quality first
test_url = "https://youtu.be/kopoLzvh5jY?si=CwgrjehubePXI_Uw"

print("=== Checking available formats ===")
list_available_formats(test_url)

print("\n=== Downloading in highest quality ===")
# Uncomment to download:
# download_highest_quality(test_url)

print("\n=== Alternative: Download specific quality ===")
# For 4K: download_specific_quality(test_url, 2160)
download_specific_quality(test_url, 1080)
# For 720p: download_specific_quality(test_url, 720)

=== Checking available formats ===
ID      EXT   RESOLUTION FPS CH |    FILESIZE    TBR PROTO | VCODEC           VBR ACODEC      ABR ASR MORE INFO
----------------------------------------------------------------------------------------------------------------------------------
sb3     mhtml 48x27        1    |                    mhtml | images                                   storyboard
sb2     mhtml 80x45        1    |                    mhtml | images                                   storyboard
sb1     mhtml 160x90       1    |                    mhtml | images                                   storyboard
sb0     mhtml 320x180      1    |                    mhtml | images                                   storyboard
233     mp4   audio only        |                    m3u8  | audio only           unknown             [en] Untested, Default, low
234     mp4   audio only        |                    m3u8  | audio only           unknown             [en] Untested, Default, high
249-drc w

{'id': 'kopoLzvh5jY',
 'title': 'Multi-Agent Hide and Seek',
 'formats': [{'format_id': 'sb3',
   'format_note': 'storyboard',
   'ext': 'mhtml',
   'protocol': 'mhtml',
   'acodec': 'none',
   'vcodec': 'none',
   'url': 'https://i.ytimg.com/sb/kopoLzvh5jY/storyboard3_L0/default.jpg?sqp=-oaymwENSDfyq4qpAwVwAcABBqLzl_8DBgjvh8yoBg==&sigh=rs$AOn4CLB7qgCE2toFUG3Siq1tyhYYZ4hshA',
   'width': 48,
   'height': 27,
   'fps': 0.5649717514124294,
   'rows': 10,
   'columns': 10,
   'fragments': [{'url': 'https://i.ytimg.com/sb/kopoLzvh5jY/storyboard3_L0/default.jpg?sqp=-oaymwENSDfyq4qpAwVwAcABBqLzl_8DBgjvh8yoBg==&sigh=rs$AOn4CLB7qgCE2toFUG3Siq1tyhYYZ4hshA',
     'duration': 177.0}],
   'audio_ext': 'none',
   'video_ext': 'none',
   'vbr': 0,
   'abr': 0,
   'tbr': None,
   'resolution': '48x27',
   'aspect_ratio': 1.78,
   'filesize_approx': None,
   'http_headers': {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.164 Safari/537.

In [2]:
import yt_dlp
import os

# DIAGNOSTIC FUNCTION - Check what qualities are actually available
def check_video_qualities(url):
    """
    Detailed analysis of available video qualities
    """
    ydl_opts = {'quiet': True}
    
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=False)
            
            print(f"Video: {info.get('title', 'Unknown')}")
            print("=" * 60)
            
            formats = info.get('formats', [])
            
            # Separate video and audio formats
            video_formats = [f for f in formats if f.get('vcodec', 'none') != 'none']
            audio_formats = [f for f in formats if f.get('acodec', 'none') != 'none']
            
            print(f"\n📹 VIDEO FORMATS AVAILABLE ({len(video_formats)} found):")
            print("-" * 50)
            
            # Sort by quality (height)
            video_formats.sort(key=lambda x: (x.get('height', 0), x.get('width', 0)), reverse=True)
            
            for fmt in video_formats[:15]:  # Show top 15 video formats
                format_id = fmt.get('format_id', 'N/A')
                height = fmt.get('height', 'N/A')
                width = fmt.get('width', 'N/A')
                fps = fmt.get('fps', 'N/A')
                ext = fmt.get('ext', 'N/A')
                filesize = fmt.get('filesize')
                vcodec = fmt.get('vcodec', 'N/A')
                
                resolution = f"{width}x{height}" if width != 'N/A' and height != 'N/A' else 'N/A'
                size_mb = f"{filesize // (1024*1024)} MB" if filesize else "Unknown"
                
                print(f"ID: {format_id:<8} | {resolution:<10} | {fps}fps | {ext:<4} | {vcodec:<12} | {size_mb}")
            
            print(f"\n🔊 AUDIO FORMATS AVAILABLE ({len(audio_formats)} found):")
            print("-" * 50)
            
            # Sort by audio quality (bitrate)
            audio_formats.sort(key=lambda x: x.get('abr', 0), reverse=True)
            
            for fmt in audio_formats[:10]:  # Show top 10 audio formats
                format_id = fmt.get('format_id', 'N/A')
                abr = fmt.get('abr', 'N/A')
                ext = fmt.get('ext', 'N/A')
                acodec = fmt.get('acodec', 'N/A')
                filesize = fmt.get('filesize')
                
                size_mb = f"{filesize // (1024*1024)} MB" if filesize else "Unknown"
                
                print(f"ID: {format_id:<8} | {abr} kbps | {ext:<4} | {acodec:<8} | {size_mb}")
            
            # Show the best combined formats
            print(f"\n🎯 RECOMMENDED FORMAT STRINGS:")
            print("-" * 50)
            
            if video_formats and audio_formats:
                best_video_id = video_formats[0].get('format_id')
                best_audio_id = max(audio_formats, key=lambda x: x.get('abr', 0)).get('format_id')
                
                print(f"Highest quality: 'bestvideo+bestaudio' or '{best_video_id}+{best_audio_id}'")
                print(f"4K if available: 'bestvideo[height<=2160]+bestaudio'")
                print(f"1080p: 'bestvideo[height<=1080]+bestaudio'") 
                print(f"720p: 'bestvideo[height<=720]+bestaudio'")
            
            return info
            
    except Exception as e:
        print(f"Error checking qualities: {str(e)}")
        return None

# Run this to see exactly what's available for your video
print("🔍 ANALYZING AVAILABLE QUALITIES...")
check_video_qualities("https://youtu.be/kopoLzvh5jY?si=CwgrjehubePXI_Uw")

🔍 ANALYZING AVAILABLE QUALITIES...
Video: Multi-Agent Hide and Seek

📹 VIDEO FORMATS AVAILABLE (37 found):
--------------------------------------------------
ID: 625      | 3840x2160  | 30.0fps | mp4  | vp09.00.50.08 | Unknown
ID: 313      | 3840x2160  | 30fps | webm | vp9          | 320 MB
ID: 401      | 3840x2160  | 30fps | mp4  | av01.0.12M.08 | 191 MB
ID: 620      | 2560x1440  | 30.0fps | mp4  | vp09.00.50.08 | Unknown
ID: 271      | 2560x1440  | 30fps | webm | vp9          | 119 MB
ID: 400      | 2560x1440  | 30fps | mp4  | av01.0.12M.08 | 80 MB
ID: 270      | 1920x1080  | 30.0fps | mp4  | avc1.640028  | Unknown
ID: 137      | 1920x1080  | 30fps | mp4  | avc1.640028  | 42 MB
ID: 614      | 1920x1080  | 30.0fps | mp4  | vp09.00.40.08 | Unknown
ID: 248      | 1920x1080  | 30fps | webm | vp9          | 26 MB
ID: 399      | 1920x1080  | 30fps | mp4  | av01.0.08M.08 | 23 MB
ID: 232      | 1280x720   | 30.0fps | mp4  | avc1.4D401F  | Unknown
ID: 136      | 1280x720   | 30fps | mp4  | av

## 🚀 Getting Better Quality Downloads

### The Problem
The issue with low quality is usually caused by:
1. **Wrong format selection** - Using `'best'` might select a low-res format
2. **Combined vs Separate streams** - Modern YouTube has video and audio as separate files
3. **Not specifying the right codec** - Some formats are lower quality

### ✅ Solutions for Higher Quality

**1. Use the new `download_highest_quality()` function above**
- Uses `'bestvideo+bestaudio'` format
- Automatically merges video and audio
- Gets the absolute best quality available

**2. Check available qualities first**
- Run `check_video_qualities(url)` to see what's actually available
- Look for the highest resolution in the video formats list

**3. Use specific format strings**
```python
# For 4K (2160p):
'bestvideo[height<=2160]+bestaudio'

# For 1080p Full HD:
'bestvideo[height<=1080]+bestaudio'

# For best quality regardless:
'bestvideo+bestaudio/best'
```

### 🎯 Recommended Actions:
1. **Run cell 8** (the diagnostic function) to see available qualities
2. **Run cell 7** with `download_highest_quality()` for best results  
3. If still low quality, the video might only be available in low resolution

### Common Quality Issues:
- **Old videos**: May only be available in 480p or lower
- **Live streams**: Often limited quality options
- **Region restrictions**: May force lower quality
- **Copyright content**: Sometimes limited to lower resolutions